In [1]:
#!/usr/bin/env python3
"""
Этап 3: Генерация химерной последовательности
"""

import random
import math

def clean_sequence(sequence):
    """
    Очищает последовательность от нестандартных нуклеотидов (N и др.)
    Возвращает только A, T, G, C
    """
    valid_bases = {'A', 'T', 'G', 'C'}
    cleaned = [base for base in sequence if base in valid_bases]
    return ''.join(cleaned)

def generate_chimera(seq1, seq2, total_length=10000, mean_length=300):
    """
    Генерирует химерную последовательность
    
    Параметры:
    - seq1, seq2: исходные последовательности
    - total_length: общая длина химеры
    - mean_length: средняя длина фрагментов
    
    Возвращает:
    - chimera: химерная последовательность
    - fragments: список фрагментов (начало, конец, источник)
    """
    
    print(f"Генерация химерной последовательности длиной {total_length} bp...")
    print(f"Средняя длина фрагментов: {mean_length} bp")
    
    # Очищаем последовательности
    seq1_clean = clean_sequence(seq1)
    seq2_clean = clean_sequence(seq2)
    
    print(f"Очищенная seq1: {len(seq1_clean)} bp (исходно: {len(seq1)} bp)")
    print(f"Очищенная seq2: {len(seq2_clean)} bp (исходно: {len(seq2)} bp)")
    
    if len(seq1_clean) < 1000 or len(seq2_clean) < 1000:
        raise ValueError("Очищенные последовательности слишком короткие")
    
    chimera_parts = []
    fragments_info = []
    current_length = 0
    current_source = 1  # 1 = seq1, 2 = seq2
    
    # Начинаем с первого источника
    while current_length < total_length:
        # Генерируем длину фрагмента из экспоненциального распределения
        # expovariate(1/mean) дает экспоненциальное с данным средним
        fragment_length = int(random.expovariate(1/mean_length))
        
        # Ограничиваем длину фрагмента
        fragment_length = max(50, min(fragment_length, 2000))
        
        # Выбираем последовательность-источник
        if current_source == 1:
            source_seq = seq1_clean
            source_name = "seq1"
        else:
            source_seq = seq2_clean
            source_name = "seq2"
        
        # Выбираем случайную начальную позицию (равномерно по длине)
        if len(source_seq) > fragment_length:
            start_pos = random.randint(0, len(source_seq) - fragment_length)
        else:
            # Если фрагмент длиннее последовательности, берем всю
            start_pos = 0
            fragment_length = len(source_seq)
        
        # Извлекаем фрагмент
        fragment = source_seq[start_pos:start_pos + fragment_length]
        
        # Если достигли нужной длины, обрезаем последний фрагмент
        if current_length + len(fragment) > total_length:
            remaining = total_length - current_length
            fragment = fragment[:remaining]
            fragment_length = remaining
        
        # Добавляем фрагмент
        chimera_parts.append(fragment)
        fragments_info.append({
            'start': current_length,
            'end': current_length + len(fragment),
            'source': current_source,
            'source_name': source_name,
            'original_start': start_pos,
            'original_end': start_pos + len(fragment)
        })
        
        current_length += len(fragment)
        
        # Переключаем источник для следующего фрагмента
        current_source = 3 - current_source  # 1->2, 2->1
    
    # Собираем химерную последовательность
    chimera = ''.join(chimera_parts)
    
    print(f"\nСгенерирована химерная последовательность:")
    print(f"  Общая длина: {len(chimera)} bp")
    print(f"  Количество фрагментов: {len(fragments_info)}")
    
    # Анализируем распределение фрагментов
    seq1_bp = sum(f['end'] - f['start'] for f in fragments_info if f['source'] == 1)
    seq2_bp = sum(f['end'] - f['start'] for f in fragments_info if f['source'] == 2)
    
    print(f"  BP от seq1: {seq1_bp} ({seq1_bp/len(chimera)*100:.1f}%)")
    print(f"  BP от seq2: {seq2_bp} ({seq2_bp/len(chimera)*100:.1f}%)")
    
    # Статистика по длинам фрагментов
    lengths = [f['end'] - f['start'] for f in fragments_info]
    avg_length = sum(lengths) / len(lengths)
    
    print(f"  Средняя длина фрагмента: {avg_length:.1f} bp")
    print(f"  Минимальная длина: {min(lengths)} bp")
    print(f"  Максимальная длина: {max(lengths)} bp")
    
    return chimera, fragments_info

# Читаем сохраненные последовательности
with open("chromosome1.fasta", "r") as f:
    lines = f.readlines()
    chromosome1_header = lines[0].strip()
    chromosome1_seq = ''.join(line.strip() for line in lines[1:])

with open("chromosome2.fasta", "r") as f:
    lines = f.readlines()
    chromosome2_header = lines[0].strip()
    chromosome2_seq = ''.join(line.strip() for line in lines[1:])

print("=" * 60)
print("ГЕНЕРАЦИЯ ХИМЕРНОЙ ПОСЛЕДОВАТЕЛЬНОСТИ")
print("=" * 60)

# Генерируем химеру
chimera_seq, fragments = generate_chimera(
    chromosome1_seq, 
    chromosome2_seq,
    total_length=10000,  # Несколько тысяч букв
    mean_length=300
)

# Сохраняем химерную последовательность
with open("chimera.fasta", "w") as f:
    f.write(">Chimera_sequence_S_griesus_M_genitalium\n")
    for i in range(0, len(chimera_seq), 70):
        f.write(chimera_seq[i:i+70] + "\n")

# Сохраняем информацию о фрагментах
with open("chimera_fragments.txt", "w") as f:
    f.write("start\tend\tsource\toriginal_start\toriginal_end\n")
    for frag in fragments:
        f.write(f"{frag['start']}\t{frag['end']}\t{frag['source']}\t")
        f.write(f"{frag['original_start']}\t{frag['original_end']}\n")

# Сохраняем истинные метки для каждой позиции
with open("chimera_true_labels.txt", "w") as f:
    for frag in fragments:
        for pos in range(frag['start'], frag['end']):
            f.write(f"{frag['source']}\n")

print("\nФайлы сохранены:")
print("  - chimera.fasta: химерная последовательность")
print("  - chimera_fragments.txt: информация о фрагментах")
print("  - chimera_true_labels.txt: истинные метки для каждой позиции")

ГЕНЕРАЦИЯ ХИМЕРНОЙ ПОСЛЕДОВАТЕЛЬНОСТИ
Генерация химерной последовательности длиной 10000 bp...
Средняя длина фрагментов: 300 bp
Очищенная seq1: 4602429 bp (исходно: 4602429 bp)
Очищенная seq2: 8317371 bp (исходно: 8317371 bp)

Сгенерирована химерная последовательность:
  Общая длина: 10000 bp
  Количество фрагментов: 32
  BP от seq1: 4975 (49.8%)
  BP от seq2: 5025 (50.2%)
  Средняя длина фрагмента: 312.5 bp
  Минимальная длина: 50 bp
  Максимальная длина: 1618 bp

Файлы сохранены:
  - chimera.fasta: химерная последовательность
  - chimera_fragments.txt: информация о фрагментах
  - chimera_true_labels.txt: истинные метки для каждой позиции
